### The basic setup

In [1]:
import os
import tempfile
import zipfile
from typing import Union

import census
import dotenv
import folium
import geopandas as gpd
import pandas as pd
import requests
import us

dotenv.load_dotenv()

%matplotlib inline

In [2]:
# Create a handler
c = census.Census(os.environ['CENSUS_API_KEY'])

In [3]:
## Grab shapefiles for tracts
def get_shapefile(state_fips: str,
                  filename: str,
                  level: str = 'TRACT',
                  year: Union[str, int] = 2018):
    """
    Get a shapefile from the Census's archive and place it in `filename`.
    
    Args:
        state_fips: The FIPS code of the state to pull the shapefiles for
        filename: The filename to save the shapefile to
        level: What level to pull. Tested on:
            * TRACT
            * COUNTY
            * BG (Block Group)
        year: The year that we will pull shapefiles from
    """
    url = 'https://www2.census.gov/geo/tiger/TIGER{year}/{level}/tl_{year}_{fips}_{lower_level}.zip'.format(
        level=level.upper(),
        lower_level=level.lower(),
        year=year,
        fips=state_fips
    )
    resp = requests.get(url, stream=True)
    resp.raise_for_status()
    with open(filename, 'wb') as f:
        for block in resp.iter_content(1024):
            f.write(block)

def open_zipped_shapefile(zipped_shapefile: str) -> gpd.GeoDataFrame:
    """
    Load a shapefile that is zipped into a GeoDataFrame.
    
    Args:
        zipped_shapefile: The location of the shapefile to open
    
    Returns:
        The GeoDataFrame representing this shapefile
    """
    with tempfile.TemporaryDirectory() as tmpdir:
        with zipfile.ZipFile(zipped_shapefile, 'r') as zipref:
            zipref.extractall(tmpdir)
        return gpd.read_file(tmpdir)

In [4]:
# Some data we'll plot may be interested in:
variables = [
    'NAME',         # Name of geography
    'B01001_001E',  # Total population
    'B19013_001E',  # Household median income
]

In [5]:
get_shapefile(us.states.RI.fips, 'ri_tracts.zip')
get_shapefile(us.states.RI.fips, 'ri_bgs.zip', level='BG')
tract_df = open_zipped_shapefile('ri_tracts.zip')
bg_df = open_zipped_shapefile('ri_bgs.zip')

In [6]:
# Create county and state boundaries
county_df = tract_df.dissolve(by='COUNTYFP')
state_df = county_df.dissolve(by='STATEFP')

### Using shapefiles to enumerate FIPS codes and plotting data

In [ ]:
# Get the collection of county FIPS from the tract shapefile
state_county_fips = tract_df[['STATEFP', 'COUNTYFP']].drop_duplicates().values

# Pull all the data from 
data = []
for state_fips, county_fips in state_county_fips:
    pulled = c.acs5.state_county_tract(variables, state_fips, county_fips, '*')
    data.extend(pulled)
    

In [ ]:
data_df = pd.DataFrame.from_records(data)
for col in data_df.columns:
    # Convert the data columns to floats
    if col.startswith('B'):
        data_df[col] = data_df[col].astype(float)

# Concatenate the state, county, and tract FIPS to create the full GEOID
# so we can join against the shapefile
data_df['geoid'] = data_df.state + data_df.county + data_df.tract

In [ ]:
data_df.head()

In [ ]:
# Merge the shapefiles and the data
tract_data = tract_df.merge(data_df, left_on='GEOID', right_on='geoid')

In [ ]:
# Plot the population!
# Note that there are tracts that have 0 population (they're water). Exclude them.
tract_data[all_data.B01001_001E > 0].plot('B01001_001E', legend=True)

### Making interactive maps with Folium

In [ ]:
# Make a constant marker so our choropleth will work
state_df['marker'] = 1

In [ ]:
# Plot the state outline
m = folium.Map(
    location=[41.6886, -71.5642],  # Coventry
    tiles='Stamen Toner',
    zoom_start=10,
    control_scale=True,
    prefer_canvas=True
)

folium.Choropleth(
    geo_data=state_df,
    name='choropleth',
    data=state_df,
    key_on='feature.properties.GEOID',
    columns=['GEOID', 'marker'],
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2
).add_to(m)

m.save('state.html')

In [ ]:
# Aggregate tract data into counties
county_data = tract_data[['geometry', 'COUNTYFP', 'B01001_001E']]\
                .dissolve(by='COUNTYFP', aggfunc='sum')\
                .reset_index()

In [ ]:
# Plot the county outlines and their populations

m = folium.Map(
    location=[41.6886, -71.5642],  # Coventry
    tiles='Stamen Toner',
    zoom_start=10,
    control_scale=True,
    prefer_canvas=True
)

folium.Choropleth(
    geo_data=county_data,
    name='choropleth',
    data=county_data,
    key_on='feature.properties.COUNTYFP',
    columns=['COUNTYFP', 'B01001_001E'],
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2
).add_to(m)

m.save('county.html')

In [ ]:
# Plot the tract populations

m = folium.Map(
    location=[41.6886, -71.5642],  # Coventry
    tiles='Stamen Toner',
    zoom_start=10,
    control_scale=True,
    prefer_canvas=True
)

folium.Choropleth(
    geo_data=all_data,
    name='choropleth',
    data=all_data,
    key_on='feature.properties.GEOID',
    columns=['GEOID', 'B01001_001E'],
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2
).add_to(m)

m.save('tract.html')

In [ ]:
# Repeat the above process for *block groups*
state_county_fips = tract_df[['STATEFP', 'COUNTYFP']].drop_duplicates().values

# Pull all the data from 
data = []
for state_fips, county_fips in state_county_fips:
    pulled = c.acs5.state_county_blockgroup(variables, state_fips, county_fips, '*')
    data.extend(pulled)

data_df = pd.DataFrame.from_records(data)
for col in data_df.columns:
    # Convert the data columns to floats
    if col.startswith('B'):
        data_df[col] = data_df[col].astype(float)

# Concatenate the state, county, and tract FIPS to create the full GEOID
# so we can join against the shapefile
data_df['geoid'] = data_df.state + data_df.county + data_df.tract + data_df['block group']

# Merge the shapefiles and the data
bg_data = bg_df.merge(data_df, left_on='GEOID', right_on='geoid')

In [ ]:
# Plot population at block group level

m = folium.Map(
    location=[41.6886, -71.5642],  # Coventry
    tiles='Stamen Toner',
    zoom_start=10,
    control_scale=True,
    prefer_canvas=True
)

folium.Choropleth(
    geo_data=bg_data,
    name='choropleth',
    data=bg_data,
    key_on='feature.properties.GEOID',
    columns=['GEOID', 'B01001_001E'],
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2
).add_to(m)

m.save('bg.html')